In [2]:
!mamba  install -c bioconda scanpy -y
!mamba install -c bioconda scrublet -y



                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.3.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['scanpy']

[+] 0.0s
bioconda/linux-64    ━━━━━╸━━━━━━━━━━━━━━━╸━   0.0 B /  ??.?MB @  ??.?MB/s  0.0

In [24]:
!mamba install -c bioconda pybiomart -y 


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.3.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['pybiomart']

[+] 0.0s
[+] 0.1s
bioconda/linux-64    ━━━━━━━━━━━━━╸━━━━━━━━━   0.0 B /  ??.?MB @  ?

In [2]:
pip install 'matplotlib == 3.6'       # fixing scanpy error

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 23.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.0
    Uninstalling matplotlib-3.7.0:
      Successfully uninstalled matplotlib-3.7.0
ERROR: Could not install packages due to an OSError: [Errno 16] Device or resource busy: '.nfs0000001247da3f500001a50c'

Note: you may need to restart the kernel to use updated packages.


,file_path,sample,lane
0,/n/scratch3/users/l/lf114/guillaume_perturb_da...,Guide,2
1,/n/scratch3/users/l/lf114/guillaume_perturb_da...,scRNA,1
2,/n/scratch3/users/l/lf114/guillaume_perturb_da...,scRNA,1
3,/n/scratch3/users/l/lf114/guillaume_perturb_da...,scRNA,2
4,/n/scratch3/users/l/lf114/guillaume_perturb_da...,Guide,1


In [3]:
%%writefile filtering_and_lane_merging.py
import numpy as np
import scanpy as sc
import anndata
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import scrublet as scr
import re
from tqdm import tqdm
import os
import pandas as pd
from glob import glob
import pathlib
import gc
import argparse
import seaborn as sns
from anndata import AnnData, read_h5ad



def create_case_doesnt_exist(x):
    if not os.path.exists(x):
       os.makedirs(x)



def analyze_batch(scRNA_ann_FILE,
                  guide_ann_FILE,
                  BATCH_NUMBER,
                  OUT_DIR_FIGURES,
                  EXPECTED_NUMBER_OF_CELLS,
                  SPECIE_MITO,
                  MITO_PERCENTAGE_ALLOWED ):
    #!mkdir $OUT_DIR_FIGURES
    create_case_doesnt_exist(OUT_DIR_FIGURES)
    # Processing scRNA_seq
    adata = anndata.read(scRNA_ann_FILE)
    # Perform SVD
    tsvd = TruncatedSVD(n_components=2)
    tsvd.fit(adata.X)
    X = tsvd.transform(adata.X)
    # Plot the cells in the 2D PCA projection
    fig, ax = plt.subplots(figsize=(10, 7))
    ax.scatter(X[:,0], X[:,1], alpha=0.5, c="green")
    #plt.axis('off')
    plt.savefig(OUT_DIR_FIGURES+'/'+ 'svd_batch' + '.png')
    plt.show()
    # Create a plot showing genes detected as a function of UMI counts.
    fig, ax = plt.subplots(figsize=(10, 7))
    x = np.asarray(adata.X.sum(axis=1))[:,0]
    y = np.asarray(np.sum(adata.X>0, axis=1))[:,0]
    ax.scatter(x, y, color="green", alpha=0.25)
    ax.set_xlabel("UMI Counts")
    ax.set_ylabel("Genes Detected")
    ax.set_xscale('log')
    ax.set_yscale('log')
    #ax.set_yscale('log', nonpositive='clip')

    ax.set_xlim((0.5, 4500))
    ax.set_ylim((0.5,2000))
    plt.savefig(OUT_DIR_FIGURES+'/'+ 'saturation_batch' + '.png')
    plt.show()
    #@title Threshold cells according to knee plot { run: "auto", vertical-output: true }
    cutoff = 200
    knee = np.sort((np.array(adata.X.sum(axis=1))).flatten())[::-1]
    cell_set = np.arange(len(knee))
    num_cells = cell_set[knee > cutoff][::-1][0]
    fig, ax = plt.subplots(figsize=(10, 7))
    ax.loglog(knee, cell_set, linewidth=5, color="g")
    ax.axvline(x=cutoff, linewidth=3, color="k")
    print (cutoff)
    print (num_cells)
    ax.axhline(y=num_cells, linewidth=3, color="k")
    ax.set_xlabel("UMI Counts")
    ax.set_ylabel("Set of Barcodes")
    plt.grid(True, which="both")
    plt.savefig(OUT_DIR_FIGURES+'/'+ 'knee_plot_batch' + '.png')
    plt.show()
    print(f"{num_cells:,.0f} cells passed the {cutoff} UMI threshold")

    sc.pp.filter_cells(adata, min_genes=cutoff)
    sc.pp.filter_cells(adata, min_counts=knee[EXPECTED_NUMBER_OF_CELLS])
    adata.var.index = [x.split('.')[0] for x in adata.var.index]  #removing the dot from ensembl in the ann data
    mito_ensembl_ids = sc.queries.mitochondrial_genes(SPECIE_MITO, attrname="ensembl_gene_id")
    mito_genes = mito_ensembl_ids["ensembl_gene_id"].values
    # for each cell compute fraction of counts in mito genes vs. all genes
    # the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
    adata.obs['percent_mito'] = np.sum(
        adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
    # add the total counts per cell as observations-annotation to adata
    adata.obs['n_counts'] = adata.X.sum(axis=1).A1
    sc.pl.scatter(adata, x='n_counts', y='percent_mito')
    plt.savefig(OUT_DIR_FIGURES+'/'+ 'mito_scatter_batch' + '.png')

    #We processed the cDNA UMI count
    #matrix and retained cells with less than 20% mitochondrial reads and at least 850 unique gene
    #UMIs 

    adata = adata[adata.obs.percent_mito < MITO_PERCENTAGE_ALLOWED]  
    sc.pl.violin(adata, ['n_genes', 'n_counts', 'percent_mito'], jitter=0.4, multi_panel=True)
    plt.savefig(OUT_DIR_FIGURES+'/'+ 'box_plot_batch' + '.png')
    sc.pl.highest_expr_genes(adata, n_top=20, gene_symbols='feature_name')
    plt.savefig(OUT_DIR_FIGURES+'/'+ 'top_genes_batch' + '.png')

    scrub = scr.Scrublet(adata.X)
    adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets()
    scrub.plot_histogram()
    plt.savefig(OUT_DIR_FIGURES+'/'+ 'doublets_batch' + '.png')
    sum(adata.obs['predicted_doublets'])


    adata.obs['doublet_info'] = adata.obs["predicted_doublets"].astype(str)
    adata = adata[adata.obs['predicted_doublets'] == False] # removing doublets

    # PROCESSING THE GUIDES

    adata_guide = anndata.read(guide_ann_FILE)
    adata_guide.obs['number_of_nonzero_guides'] = [ x.tolist()[0][0] for x in np.sum(adata_guide.X>0, axis=1)]

    sc.pl.violin(adata_guide, ['number_of_nonzero_guides'], jitter=0.4, multi_panel=True)
    plt.savefig(OUT_DIR_FIGURES+'/'+ 'guide_non_zero_batch' +'.png')

    #getting intersection

    cell_bar_intersection = set(adata_guide.obs.index.values).intersection(adata.obs.index.values)


    adata = adata[[ c in cell_bar_intersection for c in adata.obs.index ]] #filtering not shared barcodes (guide and scrna)

    adata_guide = adata_guide[[ c in cell_bar_intersection for c in adata_guide.obs.index ]] #filtering not shared barcodes (guide and scrna)

    adata.obs['batch_number'] = BATCH_NUMBER
    adata_guide.obs['batch_number'] = BATCH_NUMBER
    return adata,adata_guide



def preparing_files_initialing(path_list):
    
    print ('preparing files...')
    # create_main_fig_dir = 'results_per_lane'
    # create_case_doesnt_exist(create_main_fig_dir)
    # #capturing files generated (guides and ScRNAseq)
    # #how to capture the file names?
    # assert len(set([f.parent.parent.name for f in path_objects_test])) == len(path_objects_test), 'This seems like multiple runs inside the same directory, use resume to avoid it or change the directory'
    dir_fastqz = [f + '/counts_unfiltered/adata.h5ad' for f in path_list]
    df_files = pd.DataFrame(dir_fastqz, columns=['file_path'])
    df_files['sample'] = df_files['file_path'].apply(lambda x : 'Guide' if 'guide' in x else 'scRNA' )
    df_files['lane'] =   df_files['file_path'].apply(lambda x : x.split('/')[-3].split('_')[1].replace('L', '') )  # check if how it will be handled in the future
    
    # print (len(dir_fastqz))
    # print (df_files.shape)
    # print (df_files)
    return df_files



def preparing_files(path):
    create_main_fig_dir = 'results_per_lane'
    create_case_doesnt_exist(create_main_fig_dir)
    #capturing files generated (guides and ScRNAseq)
    #how to capture the file names?
    df_files = pd.read_csv(path, sep='\t')
    # path_objects_test = [f for f in pathlib.Path(path).rglob("*") if 'h5ad' in f.name]
    # assert len(set([f.parent.parent.name for f in path_objects_test])) == len(path_objects_test), 'This seems like multiple runs inside the same directory, use resume to avoid it or change the directory'
    # dir_fastqz = [str(f) for f in path_objects_test]
    # df_files = pd.DataFrame(dir_fastqz, columns=['file_path'])
    # df_files['sample'] = df_files['file_path'].apply(lambda x : 'Guide' if 'guide' in x else 'scRNA' )
    # df_files['lane'] =   df_files['file_path'].apply(lambda x : x.split('/')[-3].split('_')[1].replace('L', '') )  # check if how it will be handled in the future
    return df_files
    

    
    
    
def execute_analysis(df_files, EXPECTED_CELL_NUMBER, MITO_SPECIE, MITO_EXPECTED_PERCENTAGE ):
    
    ann_scrna_data_to_concat = []
    ann_guide_data_to_concat = []
    
    
    for k, v in df_files.groupby('lane'):
        lane_n = v.query('sample== "Guide" ')['lane'].values[0]
        file_guide = v.query('sample== "Guide" ')['file_path'].values[0]
        file_scRNA = v.query('sample== "scRNA" ')['file_path'].values[0]
        out_name = f'results_per_lane/lane_{lane_n}'
        exp_number_of_cells = EXPECTED_CELL_NUMBER
        specie_mito = MITO_SPECIE
        mito_percent_allowed = MITO_EXPECTED_PERCENTAGE
        print (out_name)
        ann_scrna, ann_guide = analyze_batch(scRNA_ann_FILE = file_scRNA,
                      guide_ann_FILE = file_guide,
                      BATCH_NUMBER = lane_n,
                      OUT_DIR_FIGURES = out_name,
                      EXPECTED_NUMBER_OF_CELLS = exp_number_of_cells,
                      SPECIE_MITO = specie_mito,
                      MITO_PERCENTAGE_ALLOWED = mito_percent_allowed
                     )
        ann_scrna_data_to_concat.append(ann_scrna)
        ann_guide_data_to_concat.append(ann_guide)
        
        
    return ann_scrna_data_to_concat, ann_guide_data_to_concat



def concact_lanes(ann_scrna_data_to_concat, ann_guide_data_to_concat):
    
    #Concact batch results in a single ANN data file.  
    print ('Concacting lanes...')
    concat_scrna_ann = anndata.concat(ann_scrna_data_to_concat, merge="same")
    concat_scrna_ann.obs_names_make_unique()
    concat_guide_ann = anndata.concat(ann_guide_data_to_concat, merge="same")
    concat_guide_ann.obs_names_make_unique()
    return concat_scrna_ann, concat_guide_ann


def filtering_low_expressed_genes(concat_scrna_ann,PERCENTAGE_OF_CELLS_TO_INCLUDE_TRANSCRIPT = 0.01):
    ### Filtering lower represented transcripts  
    #- Filter transcripts not presented on at least 1% of all cells (all lanes)
    #default  #  1%
    print ('Filtering Low expressed genes...')
    MINIMAL_CELLS_TO_INCLUDE_TRANSCRIPT = int(concat_scrna_ann.shape[0] * PERCENTAGE_OF_CELLS_TO_INCLUDE_TRANSCRIPT)
    sc.pp.filter_genes(concat_scrna_ann, min_cells=MINIMAL_CELLS_TO_INCLUDE_TRANSCRIPT) #remove genes not present more than > 1% of all cells  
    



def saving_anndata_files( concat_scrna_ann,  concat_guide_ann ):
    print ('saving ann data...')
    concat_scrna_ann.write(f'results_per_lane/full_raw_scrna_ann_data.h5ad')
    concat_guide_ann.write(f'results_per_lane/full_raw_guide_ann_data.h5ad')
    
    

def merge_data(concat_guide_ann, MERGE=False):
    '''
        Detecting guides targeting the same region.
        Merge the guide counts (sum operation)
        Change the name for an unique target (removing the |1 and |2)
    '''
    print ('Merging guides...')
    
    guides_to_merge = pd.DataFrame.sparse.from_spmatrix(concat_guide_ann.X).T
    if MERGE:
      guides_to_merge['guide'] = [f.split('|')[0] for f in concat_guide_ann.var['feature_name'].values]
    else:
      guides_to_merge['guide'] = [f for f in concat_guide_ann.var['feature_name'].values]

    guides_to_merge_numpy = guides_to_merge[guides_to_merge.columns[:-1]].values  # creating a numpy and removing the last column
    grouped_version = guides_to_merge[[0, 'guide']].groupby('guide')
    guides_to_merge_out = pd.DataFrame([ guides_to_merge_numpy[v.index.values].sum(0) for k,v in grouped_version])
    guides_to_merge_out.index = list(grouped_version.groups.keys())
    guides_to_merge_out.columns = concat_guide_ann.obs.index.values
    return guides_to_merge_out
    
    
    
def binaryzing_matrix(guides_to_merge_out, GUIDE_UMI_LIMIT=5 ):
    
    print ('Binarizing guide')
    guide_matrix = guides_to_merge_out
    merged_guide_matrix_binary   =  (guide_matrix > GUIDE_UMI_LIMIT) * 1
    return merged_guide_matrix_binary


def formating_exp_matrix(concat_scrna_ann):
    print ('Formating scRNAseq matrix...')
    Gene_matrix  = pd.DataFrame.sparse.from_spmatrix(concat_scrna_ann.X).T #takes a few seconds
    Gene_matrix.columns = concat_scrna_ann.obs.index.values
    Gene_matrix.index = concat_scrna_ann.var['feature_name'].values
    return Gene_matrix
    
   
def generating_covariates(concat_scrna_ann, guide_matrix):
    '''
    percent_mito::Cell quality
    log_number_of_detected_genes : Sequencing depht (dropouts)
    log_total_gene_count: Sequencing depht Transcripts
    log_total_guide_count : Can be generated by low infectinon rate
    '''
    print ('Generating covariate matrix')
    covariate_matrix = pd.DataFrame(concat_scrna_ann.obs['batch_number'].values, columns=['bath_number'])
    covariate_matrix.index = concat_scrna_ann.obs.index.values
    covariate_matrix['percent_mito'] = concat_scrna_ann.obs['percent_mito'].values
    covariate_matrix['log_number_of_detected_genes'] = np.log(concat_scrna_ann.obs['n_genes'].values)
    covariate_matrix['log_total_gene_count'] = np.log(concat_scrna_ann.obs['n_genes'].values +1)
    covariate_matrix['log_total_guide_count'] = np.log(guide_matrix.T.sum(1).values + 1) 
    plt.clf()
    sns.boxplot(x='bath_number',y='log_number_of_detected_genes', data=covariate_matrix)
    plt.title('Variation of the Log(number detected genes) among batchs ')
    plt.savefig('batch_effect_visualization.png')
    return covariate_matrix


def converting_final_table_to_anndata(Gene_matrix, merged_guide_matrix_binary, covariate_matrix):
    '''
    ## Creating a simple ANNDATA structures
    #The transcriptomes and guide matrix will generate Anndata vars and  the covariates will be stored in the anndata.obs (in both anndatas) 
    '''
    print ('Generating transcripts and guides final anndata files')
    expression = Gene_matrix.copy()
    guide = merged_guide_matrix_binary.copy()
    covariates = covariate_matrix.copy()

    guides_metadata = pd.DataFrame(guide.index.values, columns=['ID'])
    guides_metadata.index = guides_metadata['ID'] # This is necessariy for the ANN data creation

    expression = expression.T
    guide = guide.T
    
    #removing genes with duplicated names
    expression = expression.loc[:,~expression.columns.duplicated()] #remove duplicate genes
    
    ann_exp =   AnnData(X=expression, obs=covariates)
    ann_guide = AnnData(X=guide, obs=covariates)
    ann_guide.write(f'results_per_lane/processed_anndata_guides_data.h5ad')
    ann_exp.write(  f'results_per_lane/processed_anndata_transcripts_data.h5ad')
    

print ('executing')

parser = argparse.ArgumentParser(description='Description of your program')
parser.add_argument('--path', type=str,
                help='The path to the directory having the initialized df (default: current working directory)')

parser.add_argument('--expected_cell_number', type=int, default=8000,
                    help='The expected number of cells in the sample (default: 8000)')
parser.add_argument('--mito_specie', type=str, default='hsapiens',
                    help='The mitochondrial species to use for filtering (default: hsapiens)')
parser.add_argument('--mito_expected_percentage', type=float, default=0.2,
                    help='The expected percentage of mitochondrial reads (default: 0.2)')
parser.add_argument('--percentage_of_cells_to_include_transcript', type=float, default=0.01,
                    help='The percentage of cells to include in transcript counts (default: 0.01)')
parser.add_argument('--merge', action='store_true', 
                    help='Merge transcript counts from multiple runs of the pipeline')
parser.add_argument('--guide_umi_limit', type=int, default=5,
                    help='The maximum number of mismatches allowed for a guide-UMI pair (default: 5)')

args = parser.parse_args()

PATH = args.path
EXPECTED_CELL_NUMBER = args.expected_cell_number
MITO_SPECIE = args.mito_specie
MITO_EXPECTED_PERCENTAGE = args.mito_expected_percentage
PERCENTAGE_OF_CELLS_TO_INCLUDE_TRANSCRIPT = args.percentage_of_cells_to_include_transcript
MERGE= args.merge
GUIDE_UMI_LIMIT= args.guide_umi_limit

df_processed = preparing_files(PATH)

in_concact_scrna_ann, in_concact_guide_ann = execute_analysis(df_processed, EXPECTED_CELL_NUMBER, MITO_SPECIE, MITO_EXPECTED_PERCENTAGE )

concat_scrna_ann, concat_guide_ann =  concact_lanes(in_concact_scrna_ann, in_concact_guide_ann)

filtering_low_expressed_genes(concat_scrna_ann) # inplace operation

saving_anndata_files(concat_scrna_ann, concat_guide_ann )

guides_to_merge_out = merge_data(concat_guide_ann,MERGE) 

processed_guide = binaryzing_matrix(guides_to_merge_out, GUIDE_UMI_LIMIT )

gene_matrix = formating_exp_matrix(concat_scrna_ann )

covariate_matrix = generating_covariates(concat_scrna_ann, processed_guide)

converting_final_table_to_anndata(gene_matrix, processed_guide, covariate_matrix )


Overwriting filtering_and_lane_merging.py


In [32]:
!python filtering_and_lane_merging.py --path '/n/scratch3/users/l/lf114/guillaume_perturb_data/testing_w_option_4' --expected_cell_number 8000 --mito_specie hsapiens --mito_expected_percentage 0.2 --percentage_of_cells_to_include_transcript 0.2 --merge --guide_umi_limit 5


executing
results_per_lane/lane_1
filtering_and_lane_merging.py:46: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
filtering_and_lane_merging.py:61: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
200
1234
filtering_and_lane_merging.py:77: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
1,234 cells passed the 200 UMI threshold
/home/lf114/miniconda3/envs/perturbseq_pipeline/lib/python3.8/site-packages/scanpy/plotting/_utils.py:315: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  pl.show()
/home/lf114/miniconda3/envs/perturbseq_pipeline/lib/python3.8/site-packages/scanpy/plotting/_utils.py:315: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  pl.show()
/home/lf114/miniconda3

In [8]:
#guide_ann

In [9]:
#scrna_ann

In [10]:
#()dir_fastqz

results_per_lane_figures/lane_1


/tmp/ipykernel_25234/2293231456.py:26: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/tmp/ipykernel_25234/2293231456.py:41: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


200
1234


/tmp/ipykernel_25234/2293231456.py:57: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


1,234 cells passed the 200 UMI threshold


...

/tmp/ipykernel_25234/2293231456.py:57: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


1,234 cells passed the 200 UMI threshold


/home/lf114/miniconda3/envs/perturbseq_pipeline/lib/python3.8/site-packages/scanpy/plotting/_utils.py:315: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  pl.show()
/home/lf114/miniconda3/envs/perturbseq_pipeline/lib/python3.8/site-packages/scanpy/plotting/_utils.py:315: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  pl.show()
/home/lf114/miniconda3/envs/perturbseq_pipeline/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/lf114/miniconda3/envs/perturbseq_pipeline/lib/python3.8/site-packages/scanpy/plotting/_utils.py:315: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  pl.show()


Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.43
Detected doublet rate = 0.6%
Estimated detectable doublet fraction = 1.4%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 46.7%
Elapsed time: 0.4 seconds


/home/lf114/miniconda3/envs/perturbseq_pipeline/lib/python3.8/site-packages/scanpy/plotting/_utils.py:315: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  pl.show()
/tmp/ipykernel_25234/2293231456.py:112: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['batch_number'] = BATCH_NUMBER
/tmp/ipykernel_25234/2293231456.py:113: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_guide.obs['batch_number'] = BATCH_NUMBER


python: can't open file 'script.py': [Errno 2] No such file or directory


In [19]:
!pwd

/n/data1/bch/hemonc/bauer/lucassilva/guillame_perturbseq
